# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [ ]:
# Import the WeatherPy_database.csv file. 
import_file = '../Weather_Database/WeatherPy_Database.csv'
city_data_df = pd.readsv(import_file)

# Display sample data
city_data_df.head()

## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [ ]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your vacation?"))
max_temp = float(input("What is the maximum temperature you would like for your vacation?"))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [ ]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)
]
# Display sample data
preferred_cities_df.head()

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [ ]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
# Display sample data
clean_travel_cities.head()

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [ ]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[['City', 'Country', 'Max Temp', 'Desc', 'Lat', 'Lng']].copy()
hotel_df.head()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [ ]:
# Create a new column "Hotel Name".
hotel_df['Hotel Name'] = ""
# Display sample data
hotel_df.head()

## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [16]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [17]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Chuy - nearest hotel: Alerces
Itaituba - nearest hotel: TG Hotel
Hilo - nearest hotel: Dolphin Bay Hotel
Rikitea - nearest hotel: Chez Bianca & Benoit
Gat - nearest hotel: No hotel found
Santa Maria Da Vitoria - nearest hotel: Hotel Passarela
Hambantota - nearest hotel: Oasis Aurweda Resort Hambathota
Kapaa - nearest hotel: Pono Kai Resort
Ixtapa - nearest hotel: No hotel found
Codrington - nearest hotel: No hotel found
Mar Del Plata - nearest hotel: Nuevo Ostende
Mahebourg - nearest hotel: Grand Bel Air
Meulaboh - nearest hotel: Hotel Meuligou
Candolim - nearest hotel: bluesea Beach Resort
Katherine - nearest hotel: No hotel found
Bull Savanna - nearest hotel: Junction Guest House
Conceicao Do Araguaia - nearest hotel: No hotel found
Saint-Philippe - nearest hotel: Le Baril
Oriximina - nearest hotel: No hotel found
Buta - nearest hotel: Hôtel Rubi
Ulladulla - nearest hotel: Hotel Marlin
Sambava - nearest hotel: Royal Sava Hôtel
Kavaratti - nearest hotel: No hotel

,City,Country,Max Temp,Desc,Lat,Lng,Hotel Name
0,Chuy,UY,79.20,Clouds,-33.6971,-53.4616,Alerces
5,Itaituba,BR,78.67,Clouds,-4.2761,-55.9836,TG Hotel
8,Hilo,US,76.62,Rain,19.7297,-155.0900,Dolphin Bay Hotel
11,Rikitea,PF,78.93,Clouds,-23.1203,-134.9692,Chez Bianca & Benoit
16,Gat,IL,70.79,Clouds,31.6100,34.7642,No hotel found
...,...,...,...,...,...,...,...
542,Azusa,US,73.76,Clear,34.1336,-117.9076,No hotel found
544,Taltal,CL,72.45,Clouds,-25.4000,-70.4833,Hostería Taltal
549,Dargaville,NZ,71.83,Rain,-35.9333,173.8833,Central Hotel
555,Codajas,BR,78.76,Clouds,-3.8367,-62.0569,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [25]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name'] != "No hotel found")]
# Display sample data
clean_hotel_df.head()

,City,Country,Max Temp,Desc,Lat,Lng,Hotel Name
0,Chuy,UY,79.20,Clouds,-33.6971,-53.4616,Alerces
5,Itaituba,BR,78.67,Clouds,-4.2761,-55.9836,TG Hotel
8,Hilo,US,76.62,Rain,19.7297,-155.0900,Dolphin Bay Hotel
11,Rikitea,PF,78.93,Clouds,-23.1203,-134.9692,Chez Bianca & Benoit
23,Santa Maria Da Vitoria,BR,76.26,Clouds,-13.3947,-44.1886,Hotel Passarela


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [26]:
# Create the output File (CSV)
output_file = 'WeatherPy_vacation.csv'

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index_label = 'City ID')

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [31]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ['City', 'Country', 'Desc', 'Max Temp']
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Desc)